In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Dense, Flatten, InputLayer, Conv1D, MaxPool1D

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
FEATURE_LEN = 80000
INPUT_SHAPE = (FEATURE_LEN, 2)
LABEL_NUM = 3

In [4]:
dataset = np.load('./dataset_inc.npz')
features = dataset["features"].astype(np.float32) / 32767
labels = dataset["labels"]

In [40]:
def create_bi_rnn_model(feature_len, labels_num):
    model = Sequential()
    model.add(InputLayer(input_shape = (feature_len, 2)))
    # model.add(Bidirectional(LSTM(64, return_sequences = True)))
    # model.add(Dropout(0.2))
    # model.add(Bidirectional(LSTM(32, return_sequences = True)))
    model.add(Bidirectional(LSTM(32, return_sequences = False)))
    model.add(Dense(16, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(labels_num, activation="softmax"))


    model.compile(
        loss = 'categorical_crossentropy', 
        optimizer = Adam(learning_rate = 0.001), 
        metrics = ['acc']
    )
    return model

In [41]:
def create_rnn_model(feature_len, labels_num):
    model = Sequential()
    model.add(InputLayer(input_shape = (feature_len, 2)))
    # model.add(LSTM(128, return_sequences = True))
    # model.add(Dropout(0.2))
    # model.add(LSTM(64, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences = False))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(labels_num, activation="softmax"))


    model.compile(
        loss = 'categorical_crossentropy', 
        optimizer = Adam(learning_rate = 0.001), 
        metrics = ['acc']
    )
    return model

In [5]:
def create_cnn_model(feature_len, labels_num):
    model = Sequential()
    model.add(InputLayer(input_shape = (feature_len, 2)))
    model.add(Conv1D(128, (3,), activation="relu", padding="same"))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    model.add(Conv1D(64, (3,), activation="relu", padding="same"))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    model.add(Conv1D(32, (3,), activation="relu", padding="same"))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    model.add(Conv1D(16, (3,), activation="relu", padding="same"))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    model.add(Conv1D(8, (3,), activation="relu", padding="same"))
    model.add(Dropout(0.2))
    model.add(MaxPool1D(2))
    model.add(Flatten())
    model.add(Dense(labels_num, activation="softmax"))

    model.compile(
        loss = 'categorical_crossentropy', 
        optimizer = Adam(learning_rate = 0.001), 
        metrics = ['acc']
    )
    return model


In [6]:
model = create_cnn_model(FEATURE_LEN, LABEL_NUM)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 80000, 128)        896       
_________________________________________________________________
dropout (Dropout)            (None, 80000, 128)        0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 40000, 128)        0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40000, 64)         24640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 40000, 64)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20000, 64)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20000, 32)         6

In [7]:
model.fit(features[:600], labels[:600], batch_size=20, epochs=10, validation_data=(features[600:], labels[600:]))

Epoch 1/10
30/30 [==============================] - 9s 222ms/step - loss: 0.7064 - acc: 0.6917 - val_loss: 0.1410 - val_acc: 0.9808
Epoch 2/10
30/30 [==============================] - 7s 217ms/step - loss: 0.0948 - acc: 0.9900 - val_loss: 0.0835 - val_acc: 1.0000
Epoch 3/10
30/30 [==============================] - 6s 213ms/step - loss: 0.0142 - acc: 1.0000 - val_loss: 0.0224 - val_acc: 1.0000
Epoch 4/10
30/30 [==============================] - 7s 218ms/step - loss: 6.4853e-04 - acc: 1.0000 - val_loss: 0.0106 - val_acc: 1.0000
Epoch 5/10
30/30 [==============================] - 6s 215ms/step - loss: 1.8935e-04 - acc: 1.0000 - val_loss: 0.0087 - val_acc: 1.0000
Epoch 6/10
30/30 [==============================] - 6s 213ms/step - loss: 1.0955e-04 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 7/10
30/30 [==============================] - 6s 213ms/step - loss: 8.8611e-05 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 8/10
30/30 [==============================] - 6s 212ms

In [13]:
model.save("model.h5")

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

In [9]:
emnist_ds = tf.data.Dataset.from_tensor_slices((features)).batch(1)

def representative_data_gen():
    for input_value in emnist_ds.take(100):
        yield [input_value]
    
converter.representative_dataset = representative_data_gen

In [12]:
import pathlib

converted_model = converter.convert()

generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"DOA_float32.tflite"
converted_model_file.write_bytes(converted_model)

INFO:tensorflow:Assets written to: C:\Users\OAO\AppData\Local\Temp\tmpm9xv0z8i\assets


INFO:tensorflow:Assets written to: C:\Users\OAO\AppData\Local\Temp\tmpm9xv0z8i\assets


116592